In [2]:
!pip install arxiv

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [16]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [17]:
wiki_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\RISHAV BHATTACHARJEE\\anaconda3\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [18]:
wiki_tool.name

'wikipedia'

## Create custom tool

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/tracing")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002064CC48A50>)

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about Langsmith. For any question about Langsmith you must use this tool")

In [10]:
retriever_tool.name

'langsmith_search'

In [15]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [19]:
tools = [wiki_tool,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\RISHAV BHATTACHARJEE\\anaconda3\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about Langsmith. For any question about Langsmith you must use this tool', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002067F7CCC20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain

In [20]:
from langchain_community.llms import Ollama
llm= Ollama(model="llama2")

In [22]:
from langchain import hub
prompt = hub.pull("tomas2x/llama2-agent-with-tools")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n    {tools}\n    Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n    Valid "action" values: "Final Answer" or {tool_names}\n    Provide only ONE action per $JSON_BLOB, as shown:\n    ```\n    {{\n      "action": $TOOL_NAME,\n      "action_input": $INPUT\n    }}\n    ```\n    Follow this format:\n    Question: input question to answer\n    Thought: consider previous and subsequent steps\n    Action:\n    ```\n    $JSON_BLOB\n    ```\n    Observation: action result\n    ... (repeat Thought/Action/Observation N times)\n    Thought: I know what to respond\n    Action:\n    ```\n    {{\n      "action": "Final Answer",\n      "action_input": "Final response to human"\n    }}\n    Begin! Reminder to ALWAYS respond with a valid j

## Agents

In [23]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

#### Agent Executer

In [24]:
from langchain.agents import AgentExecutor
agent_exec = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [25]:
agent_exec

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'tomas2x', 'lc_hub_repo': 'llama2-agent-with-tools', 'lc_hub_commit_hash': '522ec5e0893c0d9b4e2867a7ef52677eaf9f6455b44c421c1f15de84ae1cd1cd'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following t

In [26]:
agent_exec.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'tool_names', 'tools', 'chat_history'}.  Expected: ['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"